## Часть 1
Напишите скрипт, который скачивает все данные прошедших губернаторских выборов для всех избирательных участков. 

Входная точка [по ссылке](http://www.st-petersburg.vybory.izbirkom.ru/region/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222). Затем нужно перейти на сайты территориальных избирательных комиссий. Результаты нужно сохранить в  `cvs`-файл, `sqlite` базе данных или `parquet`-файле. В итоге должна получиться таблица с полями:
- название ТИК (территориальная избирательная комиссия, некоторое объединение избирательных участков на определенной территории)
- номер УИК (избирательный участок)
- 14 стандартных полей из итогового протокола

Скрипт должен быть в отдельном `*.py` файле или в отдельном `jupyter`-ноутбуке. 

In [43]:
from bs4 import BeautifulSoup
import requests
import csv
import re
import pandas as pd

In [44]:
#Выделим ссылки с исходного сайта:
source = requests.get("http://www.st-petersburg.vybory.izbirkom.ru/region/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222").text
soup = BeautifulSoup(source, 'lxml')

links = []
for l in soup.find_all('a', href=re.compile(r"type=222")):
    links.append(l['href'])
links, len(links)

(['http://www.st-petersburg.vybory.izbirkom.ru/region/region/st-petersburg?action=show&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217419&type=222',
  'http://www.st-petersburg.vybory.izbirkom.ru/region/region/st-petersburg?action=show&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217420&type=222',
  'http://www.st-petersburg.vybory.izbirkom.ru/region/region/st-petersburg?action=show&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217421&type=222',
  'http://www.st-petersburg.vybory.izbirkom.ru/region/region/st-petersburg?action=show&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217422&type=222',
  'http://www.st-petersburg.vybory.izbirkom.ru/region/region/st-petersburg?action=show&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78

In [45]:
#Будем записывать в csv файл:
csv_file = open('hw4/elections_scrape.csv', 'w')
csv_writer = csv.writer(csv_file)


In [46]:
#Подготовим названия столбцов:
response = requests.get(links[0])
soup = BeautifulSoup(response.text, 'lxml')

columns = []
for item in soup.findAll("td", style="color:black", width="50", align="left"):
    columns.append(item.text)
columns.insert(0, "ТИК")
columns.insert(1, "УИК")

csv_writer.writerow(columns)
columns, len(columns)

(['ТИК',
  'УИК',
  'Число избирателей, внесенных в список избирателей на момент окончания голосования',
  'Число избирательных бюллетеней, полученных участковой избирательной комиссией',
  'Число избирательных бюллетеней, выданных избирателям в помещении для голосования в день голосования',
  'Число избирательных бюллетеней, выданных избирателям, проголосовавшим вне помещения для голосования',
  'Число погашенных избирательных бюллетеней',
  'Число избирательных бюллетеней, содержащихся в переносных ящиках для голосования',
  'Число избирательных бюллетеней, содержащихся в стационарных ящиках для голосования',
  'Число недействительных избирательных бюллетеней',
  'Число действительных избирательных бюллетеней',
  'Число утраченных избирательных бюллетеней',
  'Число избирательных бюллетеней, не учтенных при получении',
  'Амосов Михаил Иванович',
  'Беглов Александр Дмитриевич',
  'Тихонова Надежда Геннадьевна'],
 16)

In [47]:
#Переходим по ссылкам и заносим данные в таблицу:
tik_number = 1
for l in links:
    response = requests.get(l)
    soup = BeautifulSoup(response.text, 'lxml')
    table = soup.find('table', border="0", bgcolor="#ffffff", cellpadding="2", cellspacing="1", style="width:100%;overflow:scroll", vspace="0", align="left")
    ans = []
    for tr in table.findAll('tr'):
        line = []
        for nobr in tr.findAll('nobr'):
            line.append(nobr.text)
        ans.append(line)
    ans.remove([])

    #Преобразуем строчку с УИКом к его номеру:
    for i in range(len(ans[0])):
        ans[0][i] = int(re.search(r'\d+', ans[0][i])[0])

    for i in range(len(ans[0])):
        row = [ans[j][i] for j in range(len(ans))]
        row.insert(0, tik_number)
        csv_writer.writerow(row)

    tik_number += 1